In [1]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests

Q: if they are elected, they have the metrics, if not elected, no metrics?
should I just check elected validators not all the validators?

challenge: needs rounding for the manually calcualted number, otherwise sometimes it will fail. 

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getEposMedian():
    method = "hmy_getMedianRawStakeSnapshot"
    params = []
    return get_information(method, params)['result']['epos-median-stake']

In [4]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [5]:
def M5_test():
    
    flag = True
    # get the median stake and the upper and lower level 
    median = float(getEposMedian())
    lower = median- 0.15*median
    upper = median + 0.15*median
    print("median stake is " + str(median))
    print("lower bond is " + str(lower))
    print("upper bond is " + str(upper))
    
    # check all the elected validator's effective stake
    validators = getAllValidatorInformation()
    for i in validators:
        if i['currently-in-committee'] == True:
            for s in i['metrics']['by-shard']:
                if float(s['effective-stake']) > upper or float(s['effective-stake']) < lower:
                        address = i['validator']['address']
                        bls_key = s['bls-public-key']
                        shard = s['shard-id']
                        stake = s['effective-stake']
                        print("validator: ", address, "bls public key: ", bls_key, " shard: ", shard) 
                        print("effective stake is out of range. The effective stake is ", stake)
                        print("")
                        flag = False
    if flag:
        return True
    else:
        return False

In [6]:
M5_test()

median stake is 2.96e+23
lower bond is 2.5159999999999998e+23
upper bond is 3.404e+23
validator:  one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9 bls public key:  d8b57417434b662da3171a3607fb2cf418117d22387f7fe40b7be3376d0bbd01088fd86304357bf3fba3744bc4516383  shard:  3
effective stake is out of range. The effective stake is  340411500000000000000000.000000000000000000

validator:  one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9 bls public key:  5a1d59080b9b83c6af6afb82eb7dec09dcdf8dbd3c455d027cdac2b97c80ca96b1d4009372a39b0d9f240d0b127c6283  shard:  3
effective stake is out of range. The effective stake is  340411500000000000000000.000000000000000000

validator:  one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9 bls public key:  bd8a0ab99bc6468b10992846f74b9f4c23ed1ce9a89848a4edba15bbef10906091880e9f10b3b0e53000c2b4f934cb97  shard:  3
effective stake is out of range. The effective stake is  340411500000000000000000.000000000000000000

validator:  one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9 bls public ke

False